In [1]:
# 导入本节所需要的模块
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torch.utils.data as Data
from torchvision import transforms
import hiddenlayer as hl

In [4]:
# 准备训练数据集MNIST
train_data=torchvision.datasets.MNIST(
    root="./data/FashionMNIST/MNIST",
    train=True,
    transform=transforms.ToTensor(),
    download=False
)
# 定义一个数据加载器
train_loader=Data.DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True,
    num_workers=2
)
# 准备需要使用的测试数据集
test_data=torchvision.datasets.MNIST(
    root="./data/MNIST",
    train=False,
    transform=transforms.ToTensor(),
    download=False
)
# 定义一个数据加载器
test_loader=Data.DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=True,
    num_workers=2
)

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
class RNNimc(nn.Module):
    def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
        """
        input_dim:输入数据的维度（图片每行的数据像素点）
        hidden_dim:RNN神经元的个数
        layer_dim:RNN的层数
        output_dim:隐藏层输出的维度（分类的数据）
        """
        super(RNNimc,self).__init__()
        self.hidden_dim=hidden_dim# RNN神经元个数
        self.layer_dim=layer_dim# RNN的层数
        # RNN
        self.rnn=nn.RNN(input_dim,hidden_dim,layer_dim,batch_first=True,nonlinearity='relu')
        # 连接全连接层
        self.fc1=nn.Linear(hidden_dim,output_dim)
    def forward(self,x):
        # x:[batch,time_step,input_dim]
        # 本例子time_step=图像所有像素数量/input_dim
        # out:[batch,time_step,output_siez]
        # h_n:[layer_dim,bacth,hidden_dim]
        out,h_n=self.rnn(x,None)
        # 选取最后一个时间点的out输出
        out=self.fc1(out[:,-1,:])
        return out

In [ ]:
# 模型的调用
input_dim=28 # 图片每行的像素数量
hidden_dim=128# RNN神经元个数
layer_dim=1# RNN的层数
output_dim=10# 隐藏层输出的维度（10类图像）
MyRNNimc=RNNimc(input_dim,hidden_dim,layer_dim,output_dim)
print(MyRNNimc)

In [ ]:
# 可视化卷积神经网络，输入：[batch,time_step,input_dim]
hl_graph=hl.build_graph(MyRNNimc,torch.zeros([1,28,28]))
hl_graph.theme=hl.graph.THEMES["blue"].copy()
hl_graph

In [ ]:
# 对模型进行训练
optimizer=torch.optim.RMSprop(MyRNNimc.parameters(),lr=0.0003)
criterion=nn.CrossEntropyLoss() # 交叉熵损失函数
train_loss_all=[]
train_acc_all=[]
test_loss_all=[]
test_acc_all=[]
num_epochs=30
for epoch in range(num_epochs):
    print('Epoch{}/{}'.format(epoch,num_epochs-1))
    MyRNNimc.train() # 设置模型为训练模式
    corrects=0
    train_num=0
    for step,(b_x,b_y) in enumerate(train_loader):
        # input :[bacth,time_step,input_dim]
        xdata=b_x.view(-1,28,28)
        output=MyRNNimc(xdata)
        pre_lab=torch.argmax(output,1)
        loss=criterion(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss+=loss.item()*b_x.size(0)
        corrects+=torch.sum(pre_lab==b_y.data)
        train_num+=b_x.size(0)
    # 计算进过一个epoch的训练之后在训练集上的损失和精度
    train_loss_all.append(loss/train_num)
    train_acc_all.append(corrects.double().item()/train_num)
    print('{} Train Loss: {:.4f} Train Acc: {:.4f}'
    .format(epoch,train_loss_all[-1],train_acc_all[-1]))
    # 设置模型为验证模式
    MyRNNimc.eval()
    corrects=0
    test_num=0
    for step,(b_x,b_y) in enumerate(test_loader):
        # input:[batch ,time_step, input_dim]
        xdata=b_x.viwe(-1,28,28)
        output=MyRNNimc(xdata)
        pre_lab=torch.argmax(output,1)
        loss=criterion(output,b_y)
        loss+=loss.item()*b_x.size(0)
        corrects+=torch.sum(pre_lab==b_y.data)
        test_num+=b_x.size(0)
    # 计算经过一个epoch的训练后在测试集上的损失和精度
    test_loss_all.append(loss/test_num)
    test_acc_all.append(corrects.double().item()/test_num)
    print('{} Test Loss: {:.4f} Test Acc: {:.4f}'.format(
        epoch,test_loss_all[-1],test_acc_all[-1]))


In [ ]:
# 可视化模型训练过程
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(train_loss_all,"ro-",label="Train loss")
plt.plot(test_loss_all,"bs-",label="Val loss")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("Loss")

In [ ]:
plt.subplot(1,2,2)
plt.plot(train_acc_all,"ro-",label="Train acc")
plt.plot(test_acc_all,"bs-",label="Val acc")
plt.xlabel("epoch")
plt.ylabel("acc")
plt.legend()
plt.show()